In [1]:
from music21 import converter, instrument, note, chord, stream
import numpy
from matplotlib import pyplot as plt
import keras
from sklearn.preprocessing import OneHotEncoder
from keras import Sequential, Model
from keras.layers import LSTM, Dropout, Dense, Activation, Input, concatenate
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Dropout
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Embedding
import tensorflow as tf
from tqdm import tqdm
import numpy as np

In [2]:
!unzip dataset.zip

unzip:  cannot find or open dataset.zip, dataset.zip.zip or dataset.zip.ZIP.


In [8]:
def get_notes(midi):
  notes = []

  notes_from_midi = midi.flatten().notesAndRests
  notes_sorted = sorted(notes_from_midi, key=lambda note: note.offset)

  prev_start = notes_sorted[0].offset

  for element in notes_sorted:
    if isinstance(element , note.Note):
      notes.append((float(element.offset), float(element.volume.velocity),float(element.seconds), str(element.pitch), float(element.offset-prev_start)))
    elif isinstance(element, chord.Chord):
      #notes.append((float(element.offset), float(element.volume.velocity), float(element.seconds), '+'.join(str(n) for n in element.normalOrder)))
      for noteChord in element.pitches:
        notes.append((float(element.offset),float(element.volume.velocity), float(element.seconds), str(noteChord), float(element.offset-prev_start)))

    elif isinstance(element, note.Rest):
        # Bizarre car main gauche et droite flatten (réunie)
        # Voir autre implémentation
        notes.append((float(element.offset), 0,float(element.seconds), "REST", float(element.offset-prev_start)))

    prev_start = element.offset
  return notes



In [9]:
input_note, output_note = [], []
input_offset, output_offset = [], []
input_volume, output_volume = [], []
input_duration, output_duration = [], []

note_all_songs = []
for folder in tqdm(os.walk("dataset")):
    if len(folder) != 3:
      continue
    else:
      for file in folder[2][:5]:
        file_path = str(folder[0])+"/"+file

        # convertit la piste midi
        midi = converter.parse(file_path)

        # récupère les notes, volume, dureées...
        notes = get_notes(midi)

        # ajoute à une liste globale
        note_all_songs.append(notes)

1it [00:00, 22.48it/s]


In [10]:
# fais un dataframe à partir de toutes les notes (sans distinctions)
df = pd.DataFrame([], columns=['debut_note','volume','durée(s)', 'pitch/chord', 'offset'])
df = df.reset_index(drop=True)
for elem in note_all_songs:
    df2 = pd.DataFrame(elem, columns=['debut_note','volume','durée(s)', 'pitch/chord', 'offset'])
    df2 = df2.reset_index(drop=True)
    df =pd.concat([df,df2])
    df = df.reset_index(drop=True)

In [11]:
df

,debut_note,volume,durée(s),pitch/chord,offset
0,0.0,55.0,2.50,E3,0.00
1,0.0,55.0,2.50,G3,0.00
2,0.0,55.0,1.25,E2,0.00
3,1.0,69.0,1.25,F#2,1.00
4,2.0,68.0,2.50,A3,1.00
...,...,...,...,...,...
298,86.0,74.0,3.20,G1,0.25
299,86.0,74.0,3.20,G3,0.25
300,86.0,74.0,3.20,B3,0.25
301,86.0,0.0,3.20,REST,0.00


In [12]:
#train les encoders pour les notes, volumes et durées en catégorielles (one hot)
# sur intégralité des données (obligé)
# Voir si on peut pas enregistré le transformateur
oh_notes =  OneHotEncoder().fit(df[['pitch/chord']])


# seuil/catégories modifiable
dict_volume_class = {
    "very low":10,
    "low":30,
    "low medium":50,
    "medium":70,
    "high": 90,
    "very high":128
}
bins_volume = list(dict_volume_class.values())
bins_volume.insert(0,-0.1)
values_volume = list(dict_volume_class.keys())
df["volume_class"] = pd.cut(df["volume"], bins=bins_volume, labels=values_volume)
"""df["volume_class_interval"] = pd.cut(df["volume"], bins=20)
intervals = df["volume_class_interval"]
median_values = [(interval.left + interval.right) / 2 for interval in intervals]
df["volume_class"] = median_values"""
oh_volume =  OneHotEncoder().fit(df[['volume_class']])


dict_duration_class = {
"very short" : 0.1,
"short" :0.25,
"medium":0.5,
"medium-long":0.75,
"long":1.0,
"very long": 1.5,
"super mega long":df['durée(s)'].max()
}
bins_duration = list(dict_duration_class.values())
bins_duration.insert(0,-0.1)
values_duration = list(dict_duration_class.keys())
df["duration_class"] = pd.cut(df["durée(s)"], bins=bins_duration, labels=values_duration)
"""df["duration_class_interval"] = pd.cut(df["durée(s)"], bins=20)
intervals = df["duration_class_interval"]
median_values = [(interval.left + interval.right) / 2 for interval in intervals]
df["duration_class"] = median_values"""
oh_duration =  OneHotEncoder().fit(df[['duration_class']])


In [13]:
def prepare_sequence(data, sequence_length, network_input, network_output):
  # create input sequences and the corresponding outputs
  for i in range(0, len(data) - sequence_length, 1):
      sequence_in = data[i:i + sequence_length]
      sequence_out = data[i + sequence_length]
      network_input.append(sequence_in)
      network_output.append(sequence_out)
  return (network_input, network_output)

def reshape_array_input(array_tensor, sequence_length):
  n_patterns = len(array_tensor)
  array_reshaped = numpy.reshape(array_tensor, (n_patterns, sequence_length, -1))
  return array_reshaped

def reshape_array_output(array_tensor, sequence_length):
  n_patterns = len(array_tensor)
  array_reshaped = numpy.reshape(array_tensor, (len(array_tensor), -1))
  return array_reshaped


In [14]:
sequence_length = 50

In [15]:
input_note, output_note = [], []
input_offset, output_offset = [], []
input_volume, output_volume = [], []
input_duration, output_duration = [], []
input_step, output_step = [], []

# pour chaque music, on la transforme en dataframe, transforme les notes, durées et volumes
for music in note_all_songs:
  df_music = pd.DataFrame(music, columns=['debut_note','volume','durée(s)', 'pitch/chord', 'step'])

  # encode le volume et la durée
  df_music["volume_class"] = pd.cut(df_music["volume"], bins=bins_volume, labels=values_volume)
  df_music["duration_class"] = pd.cut(df_music["durée(s)"], bins=bins_duration, labels=values_duration)


  # transform en one hot à partir des modèles entrainés
  notes_encoded = oh_notes.transform(df_music[['pitch/chord']]).toarray()
  volume_encoded = oh_volume.transform(df_music[['volume_class']]).toarray()
  duration_encoded = oh_duration.transform(df_music[['duration_class']]).toarray()

  # prépare les input (longeur = sequence_length) pour chaque morceau
  # garde certaine cohérence au morceau même
  input_offset, output_offset = prepare_sequence(df_music['debut_note'].values, sequence_length, input_offset, output_offset)
  input_note, output_note = prepare_sequence(notes_encoded, sequence_length, input_note, output_note)
  input_volume, output_volume = prepare_sequence(volume_encoded, sequence_length, input_volume, output_volume)
  input_duration, output_duration = prepare_sequence(duration_encoded, sequence_length, input_duration, output_duration)
  input_step, output_step = prepare_sequence(df_music['step'].values, sequence_length, input_step, output_step)

In [16]:
# on reshape tout
input_note_reshaped= reshape_array_input(input_note, sequence_length)
output_note_reshaped = reshape_array_output(output_note, sequence_length)

input_offset_reshaped = reshape_array_input(input_offset, sequence_length)
output_offset_reshaped = reshape_array_output(output_offset, sequence_length)

input_volume_reshaped= reshape_array_input(input_volume, sequence_length)
output_volume_reshaped = reshape_array_output(output_volume, sequence_length)

input_duration_reshaped= reshape_array_input(input_duration, sequence_length)
output_duration_reshaped = reshape_array_output(output_duration, sequence_length)


input_step_reshaped= reshape_array_input(input_step, sequence_length)
output_step_reshaped = reshape_array_output(output_step, sequence_length)


In [26]:
#création du modèle (à alléger si overfitting)
inputNotes_layer = Input(shape=(input_note_reshaped.shape[1], input_note_reshaped.shape[2]))
inputNotes = LSTM(
        8,
        input_shape=(input_note_reshaped.shape[1], input_note_reshaped.shape[2]),
        return_sequences=True
    )(inputNotes_layer)

inputVolume_layer = Input(shape=(input_volume_reshaped.shape[1], input_volume_reshaped.shape[2]))
inputVolume = LSTM(
        8,
        input_shape=(input_volume_reshaped.shape[1], input_volume_reshaped.shape[2]),
        return_sequences=True
    )(inputVolume_layer)

inputDuration_layer = Input(shape=(input_duration_reshaped.shape[1], input_duration_reshaped.shape[2]))
inputDuration = LSTM(
        8,
        input_shape=(input_duration_reshaped.shape[1], input_duration_reshaped.shape[2]),
        return_sequences=True
    )(inputDuration_layer)

inputOffset_layer = Input(shape=(input_offset_reshaped.shape[1], input_offset_reshaped.shape[2]))
inputOffset = LSTM(
        8,
        input_shape=(input_offset_reshaped.shape[1], input_offset_reshaped.shape[2]),
        return_sequences=True
    )(inputOffset_layer)

inputs = concatenate([inputNotes, inputVolume, inputDuration, inputOffset])
x = LSTM(64, return_sequences=True)(inputs)
x = Dropout(0.3)(x)
x = LSTM(64)(inputs)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)

outputNotes = Dense(8, activation='relu')(x)
outputNotes = Dense(output_note_reshaped.shape[1], activation='softmax', name="Note")(outputNotes)

outputVolume = Dense(8, activation='relu')(x)
outputVolume = Dense(output_volume_reshaped.shape[1], activation='softmax', name="Volume")(outputVolume)

outputDuration = Dense(8, activation='relu')(x)
outputDuration = Dense(output_duration_reshaped.shape[1], activation='softmax', name="Duration")(outputDuration)

outputOffset =  Dense(8, activation='relu')(x)
outputOffset = Dense(output_offset_reshaped.shape[1], activation='linear', name="Offset")(outputOffset)



model = Model(inputs=[inputNotes_layer,inputOffset_layer,  inputVolume_layer, inputDuration_layer], outputs=[outputNotes, outputOffset, outputVolume, outputDuration])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
#Adam seems to be faster than RMSProp and learns better too
model.compile(loss=["categorical_crossentropy", "mean_squared_error", "categorical_crossentropy", "categorical_crossentropy"], optimizer=optimizer)

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 27)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 50, 4)]              0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 50, 6)]              0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 50, 1)]              0         []                            
                                                                                              

In [19]:
import os

cp_callback = keras.callbacks.ModelCheckpoint(
    'model_weights_epoch.h5',  # Nom du fichier de sauvegarde avec un espace réservé pour le numéro de l'époque
    save_best_only=True,  # Sauvegarder à chaque époque, pas seulement les meilleurs modèles
    save_weights_only=True,  # Sauvegarder uniquement les poids, pas l'ensemble du modèle
    verbose=1  # Afficher un message lors de la sauvegarde
    )

In [27]:
model.fit([input_note_reshaped, input_step_reshaped, input_volume_reshaped, input_duration_reshaped], [output_note_reshaped, output_step_reshaped, output_volume_reshaped, output_duration_reshaped], epochs=400, callbacks=[cp_callback], validation_split=0.2)

Epoch 1/400
7/7 [==============================] - ETA: 0s - loss: 6.6436 - Note_loss: 3.2838 - Offset_loss: 0.4232 - Volume_loss: 1.3316 - Duration_loss: 1.6049
Epoch 1: val_loss did not improve from 5.95548
7/7 [==============================] - 12s 323ms/step - loss: 6.6436 - Note_loss: 3.2838 - Offset_loss: 0.4232 - Volume_loss: 1.3316 - Duration_loss: 1.6049 - val_loss: 6.4899 - val_Note_loss: 3.3488 - val_Offset_loss: 0.3487 - val_Volume_loss: 1.2068 - val_Duration_loss: 1.5856
Epoch 2/400
7/7 [==============================] - ETA: 0s - loss: 6.0394 - Note_loss: 3.2064 - Offset_loss: 0.3737 - Volume_loss: 1.1534 - Duration_loss: 1.3058
Epoch 2: val_loss did not improve from 5.95548
7/7 [==============================] - 0s 38ms/step - loss: 6.0394 - Note_loss: 3.2064 - Offset_loss: 0.3737 - Volume_loss: 1.1534 - Duration_loss: 1.3058 - val_loss: 6.2342 - val_Note_loss: 3.3323 - val_Offset_loss: 0.3968 - val_Volume_loss: 1.1779 - val_Duration_loss: 1.3273
Epoch 3/400
7/7 [=======

KeyboardInterrupt: ignored

In [21]:
#prend une séquence de notes, volume, durées
pattern_note = input_note[0]
pattern_offset = input_offset[0]
pattern_volume = input_volume[0]
pattern_duration = input_duration[0]
pattern_step = input_step[0]

In [22]:
pattern_step

array([0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 2.,
       2., 2., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 2., 2., 2., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.])

In [28]:
# prediction time
prediction_output = []

for i in tqdm(range(200)):

    #on reshape les input à prédire
    note_prediction_input = numpy.reshape(pattern_note, (1, len(pattern_note), -1))
    volume_prediction_input = numpy.reshape(pattern_volume, (1, len(pattern_volume), -1))
    duration_prediction_input = numpy.reshape(pattern_duration, (1, len(pattern_duration), -1))
    offset_prediction_input = numpy.reshape(pattern_offset, (1, len(pattern_offset), 1))
    step_prediction_input = numpy.reshape(pattern_step, (1, len(pattern_step), 1))

    #prédit ici
    prediction = model.predict([note_prediction_input, offset_prediction_input, volume_prediction_input, duration_prediction_input], verbose=0)

    # prédis la note en récupérant l'index max du softmax et en faisant la transofr inverse
    # à partir du one hot train sur les notes

    temperature = 1.0

    note_softmax = prediction[0]
    adjusted_softmax = note_softmax / temperature
    adjusted_softmax = adjusted_softmax / np.sum(adjusted_softmax)
    note_soft = tf.random.categorical(note_softmax, num_samples=1)[0][0]
    notes_pred = np.zeros(len(pattern_note[0]))
    #notes_pred[np.argmax(adjusted_softmax)] = 1
    notes_pred[np.argmax(prediction[0])] = 1
    #notes_pred[note_soft] = 1
    result_note = oh_notes.inverse_transform(notes_pred.reshape(1, -1))
    pattern_note = numpy.concatenate([pattern_note, [notes_pred]])
    pattern_note = pattern_note[1:]

    """
    offset_predict = prediction[1][0][0]
    pattern_offset = numpy.concatenate([pattern_offset, prediction[1][0]])
    pattern_offset = pattern_offset[1:]
    """
    step_predict = prediction[1][0][0]
    pattern_step = numpy.concatenate([pattern_step, prediction[1][0]])
    pattern_step = pattern_step[1:]

    # la même avec le volume
    volume_pred = np.zeros(len(pattern_volume[1]))
    volume_pred[np.argmax(prediction[2])] = 1
    result_volume = oh_volume.inverse_transform(volume_pred.reshape(1, -1))
    pattern_volume = numpy.concatenate([pattern_volume, [volume_pred]])
    pattern_volume = pattern_volume[1:]

    # la même avec la durée
    duration_pred = np.zeros(len(pattern_duration[0]))
    duration_pred[np.argmax(prediction[3])] = 1
    result_duration = oh_duration.inverse_transform(duration_pred.reshape(1, -1))
    pattern_duration = numpy.concatenate([pattern_duration, [duration_pred]])
    pattern_duration = pattern_duration[1:]

    # comme on prédit un mot associé au volume, on associe au mot une valeur/intensité
    #(manque des catégories)
    volume_encoded = result_volume[0][0]
    volume_decoded = dict_volume_class[volume_encoded]

    # la même avec la durée
    #(manque des catégories)
    duration_encoded = result_duration[0][0]
    duration_decoded = dict_duration_class[duration_encoded]


    #on ajoute la note, le volyme et la durée
    prediction_output.append([result_note[0][0], step_predict, volume_decoded, duration_decoded])


100%|██████████| 200/200 [00:23<00:00,  8.65it/s]


In [29]:
prediction_output

[['A3', 0.026755834, 90, 1.5],
 ['REST', 0.026755834, 90, 1.5],
 ['D5', 0.20250493, 90, 1.5],
 ['D5', 0.2569381, 90, 4.6875],
 ['F#4', 0.35265663, 90, 4.6875],
 ['A3', 0.30729347, 90, 4.6875],
 ['F#3', 0.16479316, 90, 4.6875],
 ['A3', 0.06224662, 90, 1.5],
 ['G2', 0.2072951, 10, 1.5],
 ['D5', 0.24373868, 90, 1.5],
 ['A4', 0.23869759, 90, 0.5],
 ['E4', 0.43851355, 90, 4.6875],
 ['A3', 0.24824369, 90, 4.6875],
 ['C#4', 0.22131045, 90, 4.6875],
 ['D2', 0.2351011, 90, 4.6875],
 ['REST', 0.032175288, 10, 4.6875],
 ['REST', 0.026755834, 10, 4.6875],
 ['G3', 1.8885927, 70, 4.6875],
 ['B3', 2.1477208, 90, 4.6875],
 ['E2', 0.026755834, 90, 1.5],
 ['REST', 0.026755834, 10, 1.5],
 ['F#2', 0.1914566, 90, 1.5],
 ['F#2', 0.04017391, 90, 1.5],
 ['F#2', 0.15471682, 90, 1.5],
 ['A4', 0.33771315, 90, 0.75],
 ['D5', 0.44606772, 90, 4.6875],
 ['A3', 0.40030065, 128, 4.6875],
 ['F#4', 0.38920566, 128, 4.6875],
 ['A2', 0.2517572, 90, 4.6875],
 ['A3', 0.29706195, 90, 1.5],
 ['G2', 0.55150837, 70, 1.5],
 ['A3

In [30]:
from music21.duration import Duration


In [31]:
offset = 0
output_notes = []
# on passes des prédicitons à un format écoutable à l'oreille

prev_start = 0
# pour chaque note prédite
for note_p in prediction_output:
    pattern = note_p[0]
    # si note = chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = prev_start + note_p[1]
        # met le volume associé
        new_chord.volume.velocity = note_p[2]
        # met la durée associée
        new_chord.duration = Duration(note_p[3])
        output_notes.append(new_chord)
    # si note est un rest
    elif('REST'in pattern):
      note_rest = note.Rest()
      note_rest.offset = prev_start + note_p[1]
      # durée associée (pas de volume car silencieux)
      note_rest.duration = Duration(note_p[3])
      output_notes.append(note_rest)
    else:
        new_note = note.Note(pattern)
        new_note.offset = prev_start + note_p[1]
        new_note.volume.velocity = note_p[2]
        new_note.duration = Duration(note_p[3])
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    prev_start = prev_start + note_p[1]
    # increase offset each iteration so that notes do not stack
    #offset += 0.5

In [32]:
#into midi
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_outputminecraft6.mid')

'test_outputminecraft6.mid'